# Picking the Deck to Win Pokemon Worlds 2025

### Setting the Stage
Pokemon Worlds 2025 is just around the corner and the meta game has been heating up since the release of Black Bolt and White Flare. Going into the Black Bolt and White Flare release, the three top performing decks were Raging Blot, Grimmsnarl, and Gardevior with solo Dragapult being the most common of it's variants. Newly introduced cards like [Genesect ex](https://pkmncards.com/card/genesect-ex-black-bolt-blk-067/), [Hilda](https://pkmncards.com/card/hilda-white-flare-wht-084/), and [Brave Bangle](https://pkmncards.com/card/brave-bangle-white-flare-wht-080/) and returning cards like [Air Balloon](https://pkmncards.com/card/air-balloon-black-bolt-blk-079/) have shaken up the meta significantly. 

### The Plan
To determine what deck we should pick if we were going to Worlds/what deck we think stands the strongest chance to win worlds, we will be analyzing the meta and matchups and then run some Monte Carlo simulations to get an idea of the standout deck(s). For those new to Monte Carlo simulations, it is a technique that uses random sampling to model different outcomes. In this project we will take a deck and then simulate a tournament run by randomly selecting a deck to play against every round. We then calculate the probability of the deck winning a best of 3 set against the randoly selected decks. 

To best simulate the Worlds tournament, we have to take into account the tournament structure. I will be basing the structure off of the 2024 Worlds tournament so there might be more rounds than will be in the 2025 tournament. To start, all players will play in a Day 1 8 round Swiss elimination where 6 wins is needed to make Day 2. We will run a Monte Carlo simulation of all decks for 8 rounds and determine the probability that a deck makes Day 2 of the tournament. We will then use these probabilities to construct a likely Day 2 meta. This Day 2 meta will be used to simulate the Day 2 an additional 4 rounds of swiss followed by a Top Cut of 16. From here, we can get the probablitiy that a deck will win the World Championship and then compare that to the other decks to determin which deck has the highest chance to become the Worlds winning deck. 

### Caveats
Without any official in-person tournaments, we have had to turn to online hosted tournaments on sites like [Limitless](https://play.limitlesstcg.com/) to get an idea of the meta shake up. While good for indications of the meta, caveats need to be made for conclusions made from this data. First, online tournaments can be entered by anyone whereas the Pokemon Worlds tournament is invite only with the below structure for TCG players in TPCi regions. In TPC regions, players qualified through their country's championship tournament or Master Ball League tournaments where the number of spots getting an invite varied by region. In addition to qualifying through these established means, winners of TPCi regionals or special events and top 4 at international championships recieved automatic invites that would not go against thier regions top X qualifiers. Suffice it to say, these are a the best of the best and only a small handful of those who play. I do not want to insinuate that the online players we will be basing our data on are worse than these world qualifiers, but there may be a difference in skill level which is why we will be using only data from players who made Top 16 in these online tournaments. This feels like an acceptable compromise to try and represent the caliber of players playing at Worlds. 

| Region        | Qualifiers |
|---------------|------------|
| USA & Canada  | Top 125    |
| Europe        | Top 125    |
| Latin America | Top 100    |
| Oceania       | Top 20     |
| Middle East & South Africa | Top 10 |

The second caveat we need to make is that a lot of online tournaments are best of 1 (BO1) with top cut possibly being best of 3 (BO3) if it is a larger tournament. The official Pokemon TCG format is a BO3. In our work here we can try our best to account for this but when looking at Top 16 metas, decks that perform best in BO1 might overperform when compared to the official BO3 format. 


### Now let's get into the data!
**Special thanks to [Trainer hill](https://www.trainerhill.com/), a very useful site that aggergates a lot of tournament and gives meta overviews as well as decklist analysis. Please go check them out if this stuff interests you!**

In [1]:
import pandas as pd
import numpy as np
import random

### Load Meta & Matchups

We will be using the meta and matchups of Top 16 results from online tournaments between 7/21/2025 and 8/11/2025 as a way to best represent the higher caliber players playing at Worlds

In [2]:
# Load csv files
matchups = pd.read_csv("matchups-top16.csv")
meta = pd.read_csv("meta-top16.csv")

# Create a meta dictionary for Monte Carlo to randomly choose a deck from
meta['share'] = meta['share'] / 100.0
meta_dict = dict(zip(meta['deck'], meta['share']))

### Setup our Win Probability Matrix P[Deck1,Deck2]

To build out the win probability matrix we cannot just take the `win_rate` as that would be looking at how likely the deck is to win just one singular match. Becasue the format is best of 3, we need to find the match win probability for this scenario. 

What are our possible win paths? (where $p$ is our win rate)
 1. WW  - $P_{\text{WW}} = p \times p = p^2$
 2. WLW - $P_{\text{WLW}} = p \times (1-p) \times p = p^2 (1-p)$
 3. LWW - $P_{\text{LWW}} = (1-p) \times p \times p = p^2 (1-p)$

Now we sum these scenarios to be able to calculate the probability of a match win $P_{\text{match win}}$:

$P_{\text{match win}} = P_{\text{WW}} + P_{\text{WLW}} + P_{\text{LWW}}$

$P_{\text{match win}} = p^2 + 2p^2(1-p)$

$P_{\text{match win}} = p^2 \big( 1 + 2(1-p) \big)$

$P_{\text{match win}} = p^2 \big( 3 - 2p \big)$

In scenarios where decks have not faced eachother, we are going to make the assumption that the match will be 50/50. While not realistic, it is the best we can do with the information at this time

**CAVEAT** - Ties are possible in the TCG. For the purpose of this simulation we will be treating ties as a loss. For Worlds, this is not that detrimental as there are set numbers of wins needed to get through Day 1 and it is very unlikely to win out the tournament with multiple ties.

In [3]:
# Get list of all decks
decks = sorted(set(matchups['deck1']).union(matchups['deck2']))
deck_index = {d:i for i,d in enumerate(decks)}
n = len(decks)

# Build win probability matrix P[a,b] = P(a beats b)
P = np.zeros((n,n))

for _, row in matchups.iterrows():
    a = deck_index[row['deck1']]
    b = deck_index[row['deck2']]

    # If total is NaN or zero, treat as no data
    total = row['total'] if not pd.isna(row['total']) else 0
    wins = row['wins'] if not pd.isna(row['wins']) else 0

    if total == 0:
        # No games → fall back to 50%
        p_match = 0.5
    else:
        # Calculate the match win probability
        p_win = wins / total
        p_match = p_win**2 * (3 - 2*p_win)

    P[a,b] = p_match

# mirror matches will always be 50%
for i in range(n):
    P[i,i] = 0.5

### Before Simulating, Let's Get an Idea of how Decks do Overall Meta

While this does not necessiarly tell us which deck will win in our simulations, it does give a broad overview of just how well that deck does against the meta as a whole. It will be interesting to see if there are any differences or similarites when we simulate based on meta distribution. Some decks might be getting boosted by some rare, but highly favorable matchups. 

In [4]:
def matchup_vs(deck_name, meta_dist, n):
    i = deck_index[deck_name]
    total = 0
    for opp, share in meta_dist.items():
        j = deck_index[opp]
        total += P[i,j] 
    return total /n

vs_meta = []
for deck in decks:
    win_pct = matchup_vs(deck, meta_dict, n) * 100
    vs_meta.append((deck, win_pct))

df_vs_meta = pd.DataFrame(vs_meta, columns=['Deck','Win % vs Meta'])
df_vs_meta.sort_values('Win % vs Meta', ascending=False)

,Deck,Win % vs Meta
4,dragapult-dusknoir,55.931792
11,grimmsnarl-froslass,55.109229
0,charizard-dusknoir,53.742822
9,gardevoir-ex-sv,51.940777
1,charizard-pidgeot,51.201420
3,dragapult-charizard,48.121406
5,dragapult-ex,47.566340
2,crustle-dri,47.122858
6,ethan-typhlosion,46.639733
13,joltik-box,45.639208


### Day 1

Our next step is to simulate Day 1 and return what the Day 2 meta will look like

In [5]:
def simulate_day1(deck_name, meta_dist, day1_rounds=8, min_wins=6, sims=100000):
    decks_list, fracs = zip(*meta_dist.items())
    fracs = np.array(fracs) / sum(fracs)
    qualify_count = 0
    
    for _ in range(sims):
        wins = 0
        for _ in range(day1_rounds):
            opp = random.choices(decks_list, weights=fracs, k=1)[0]
            pwin = P[deck_index[deck_name], deck_index[opp]]
            if random.random() < pwin:
                wins += 1
        if wins >= min_wins:
            qualify_count += 1
    return qualify_count / sims


# Simulate the probability of Day 2 conversion
day2_probs = {}
for deck in decks:
    day2_probs[deck] = simulate_day1(deck, meta_dict)

# Get they Day 2 meta from Day 2 conversion probabilities
day2_meta = {}
for deck in decks:
    day2_meta[deck] = meta_dict.get(deck, 0) * day2_probs[deck]

# Normalize to sum to 1
total_share = sum(day2_meta.values())
if total_share > 0:
    for deck in day2_meta:
        day2_meta[deck] /= total_share

# Convert to pandas for visualization
df_day2 = pd.DataFrame(list(day2_meta.items()), columns=['Deck', 'Meta Share'])
df_day2['Meta Share'] = df_day2['Meta Share'] * 100
df_day2.sort_values(by=['Meta Share'], ascending=False)

,Deck,Meta Share
11,grimmsnarl-froslass,22.344081
4,dragapult-dusknoir,17.124840
9,gardevoir-ex-sv,10.619395
10,gholdengo-ex,8.089174
1,charizard-pidgeot,7.637681
3,dragapult-charizard,7.508441
13,joltik-box,5.116310
6,ethan-typhlosion,4.995482
2,crustle-dri,4.874653
5,dragapult-ex,4.655703


### Day 2, Top 16, & Champion

Just like we did with Day 1, we are going to simulate Day 2, Top 16, and Win chance. In 2024, Day 2 consisted of 4 rounds of swiss with all 10+ win records continuing on to Top 16. We are going to use the same number of rounds of swiss, but with needing to get 3 wins to make Top 16. 

In [6]:
def simulate_day2(deck_name, meta_dist, swiss_rounds=4, cut_wins=3, sims=100000):
    top16_count = 0
    win_event_count = 0
    decks_list, fracs = zip(*meta_dist.items())

    for _ in range(sims):
        wins = 0
        for _ in range(swiss_rounds):
            opp = random.choices(decks_list, weights=fracs, k=1)[0]
            pwin = P[deck_index[deck_name], deck_index[opp]]
            if random.random() < pwin:
                wins += 1

        made_cut = wins >= cut_wins
        if made_cut:
            top16_count += 1
            # Simulate Top 16 (4 rounds)
            for _ in range(4):
                opp = random.choices(decks_list, weights=fracs, k=1)[0]
                pwin = P[deck_index[deck_name], deck_index[opp]]
                if random.random() >= pwin:
                    break
            else:
                win_event_count += 1

    return top16_count / sims, win_event_count / sims


results = []
for d in decks:
    p_cut, p_win = simulate_day2(d, day2_meta)
    r = {}
    r.update({'Deck': d})
    r.update({'Top 16%': p_cut})
    r.update({'Win %': p_win})
    results.append(r)

df_results = pd.DataFrame(results)
df_results['Top 16%'] = df_results['Top 16%'] * 100
df_results['Win %'] = df_results['Win %'] * 100
df_results.sort_values(by=['Win %'], ascending=False)

,Deck,Top 16%,Win %
3,dragapult-charizard,31.626,2.005
11,grimmsnarl-froslass,31.071,1.939
13,joltik-box,31.328,1.925
4,dragapult-dusknoir,29.732,1.836
0,charizard-dusknoir,27.442,1.319
1,charizard-pidgeot,25.778,1.158
2,crustle-dri,24.337,1.026
10,gholdengo-ex,24.234,1.013
9,gardevoir-ex-sv,22.792,0.824
5,dragapult-ex,15.476,0.323


### Tournament Outcome

It is impossible to say that there is one clear winner for deck choice when there is less than a 0.17% difference in chance to win Worlds between the top 4 decks. The top 4 decks, Dragapult/Charizard, Grimmsnarl, Joltik Box, and Dragapult/Dusknoir, all seem to be solid pick for this upcoming weekend.

I do think it is surprising to See Joltik Box and Dragapult/Charizard in the top choices. Their meta shares going into Day 1 were both 4.2% with a bump to 5.1% for Joltik Box and 7.5% for Dragapult/Charizard from the Day 2 conversion simulation. 

Personally, my pick for winning Worlds is Dragapult/Dusknoir. While Gardevoir was only given a 0.8% chance to win Worlds, this does not account for Day 2 Gardevoir players being some of the most skilled players having 1/3 internationals and 10/31 regional/special events this season. While Dragapult/Charizard has a better matchup into Gardevoir, I belive Dragapult/Dusknoir trades a bit off of that matchup for a better matchup against the overall field, making it my pick. 

### What if we Worlds was open and anyone could participate?

If everyone was able to particpate we can change up the code to utilize the information from all online matches. Below is the entirety of the code with results if Worlds was open to everyone.

In [7]:
# Load csv files
matchups = pd.read_csv("matchups-all.csv")
meta = pd.read_csv("meta-all.csv")

# Create a meta dictionary for Monte Carlo to randomly choose a deck from
meta['share'] = meta['share'] / 100.0
meta_dict = dict(zip(meta['deck'], meta['share']))

# Get list of all decks
decks = sorted(set(matchups['deck1']).union(matchups['deck2']))
deck_index = {d:i for i,d in enumerate(decks)}
n = len(decks)

# Build win probability matrix P[a,b] = P(a beats b)
P = np.zeros((n,n))

for _, row in matchups.iterrows():
    a = deck_index[row['deck1']]
    b = deck_index[row['deck2']]

    # If total is NaN or zero, treat as no data
    total = row['total'] if not pd.isna(row['total']) else 0
    wins = row['wins'] if not pd.isna(row['wins']) else 0

    if total == 0:
        # No games → fall back to 50%
        p_match = 0.5
    else:
        # Calculate the match win probability
        p_win = wins / total
        p_match = p_win**2 * (3 - 2*p_win)

    P[a,b] = p_match

# mirror matches will always be 50%
for i in range(n):
    P[i,i] = 0.5

vs_meta = []
for deck in decks:
    win_pct = matchup_vs(deck, meta_dict, n) * 100
    vs_meta.append((deck, win_pct))

df_vs_meta = pd.DataFrame(vs_meta, columns=['Deck','Win % vs Meta'])
df_vs_meta.sort_values('Win % vs Meta', ascending=False)

,Deck,Win % vs Meta
4,dragapult-dusknoir,48.205768
9,gardevoir-ex-sv,46.035773
1,charizard-pidgeot,45.831566
2,crustle-dri,45.399395
11,grimmsnarl-froslass,44.944723
5,dragapult-ex,44.458763
6,ethan-typhlosion,44.113884
13,joltik-box,43.429615
14,raging-bolt-ogerpon,42.303812
3,dragapult-charizard,41.841413


In [8]:

# Simulate the probability of Day 2 conversion
day2_probs = {}
for deck in decks:
    day2_probs[deck] = simulate_day1(deck, meta_dict)

# Get they Day 2 meta from Day 2 conversion probabilities
day2_meta = {}
for deck in decks:
    day2_meta[deck] = meta_dict.get(deck, 0) * day2_probs[deck]

# Normalize to sum to 1
total_share = sum(day2_meta.values())
if total_share > 0:
    for deck in day2_meta:
        day2_meta[deck] /= total_share

# Convert to pandas for visualization
df_day2 = pd.DataFrame(list(day2_meta.items()), columns=['Deck', 'Meta Share'])
df_day2['Meta Share'] = df_day2['Meta Share'] * 100
df_day2.sort_values(by=['Meta Share'], ascending=False)

,Deck,Meta Share
4,dragapult-dusknoir,13.590705
11,grimmsnarl-froslass,13.561473
10,gholdengo-ex,13.395491
9,gardevoir-ex-sv,9.625512
1,charizard-pidgeot,7.636053
6,ethan-typhlosion,6.694437
5,dragapult-ex,6.669121
14,raging-bolt-ogerpon,6.502570
3,dragapult-charizard,5.716740
13,joltik-box,5.546158


In [9]:
results = []
for d in decks:
    p_cut, p_win = simulate_day2(d, day2_meta)
    r = {}
    r.update({'Deck': d})
    r.update({'Top 16%': p_cut})
    r.update({'Win %': p_win})
    results.append(r)

df_results = pd.DataFrame(results)
df_results['Top 16%'] = df_results['Top 16%'] * 100
df_results['Win %'] = df_results['Win %'] * 100
df_results.sort_values(by=['Win %'], ascending=False)

,Deck,Top 16%,Win %
2,crustle-dri,30.634,1.837
1,charizard-pidgeot,30.289,1.775
4,dragapult-dusknoir,29.882,1.693
13,joltik-box,28.412,1.506
11,grimmsnarl-froslass,28.449,1.468
9,gardevoir-ex-sv,25.404,1.176
10,gholdengo-ex,24.841,1.084
6,ethan-typhlosion,24.413,1.078
5,dragapult-ex,24.784,1.042
7,flareon-noctowl,23.927,0.970


### Open Field Outcome

Based on the simulations, Crustle appears to take the field. It is unsurprising as in online tournaments it has great matchups into the top 2 meta share decks of Gholdengo and Grimmsnarl. It seems to be a very volatile matchup deck, either having near auto wins or near auto losses. While, I believe Crustle to be a great deck overall, I think that a lot of top players are going to be more prepared for play against it, making it a weaker pick for Worlds. 

That being said, Crustle seems like the play to bring to your local Cups and Challenges from now until the Mega Evolution set release on 9/26/2025.  